# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [10]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
import pandas as pd
from sklearn import model_selection


## Dataset

### Overview
The dataset to be used is built up from meteorological values for the city of Los Angeles, USA. It includes data like the minimum and maximum temperature on the day, the wind speed and the measured precipitation. 
The objective of this project will be to make a model capable of predicting the precipitation in the city depending on the other given meteorogical measures. For this we will use the autoML tool from Azure as well as a predifined regression model with optimized hyperparameters using HyperDrive.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [47]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Capstone_ML'

experiment=Experiment(ws, experiment_name)

found = False
key = "Precipitation-LA"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        

if not found:
        # Gets a dataset from a given URL
        example_data = 'https://raw.githubusercontent.com/RaikohGrass/ML_Azure_Capstone/main/precipitation_LA.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        df = dataset.to_pandas_dataframe()
        df.describe()
        # We drop the empty column TAVG
        dataset = dataset.drop_columns('TAVG')
        # Since we are not using deep learning or techniques that have a notion of time we can drop the date and PGTM columns
        # We also drop the station name column
        dataset = dataset.drop_columns(['STATION','NAME','DATE','PGTM'])
        #Register the dataset in the default datastore
        dataset = dataset.register(workspace=ws,
                                   name=key)




In [48]:
# Lets look at the dataframe
df.describe()

,AWND,PRCP,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08
count,1823.00,1827.00,1827.00,1827.00,1822.00,1809.00,1823.00,1809.00,551.00,39.00,861.00
mean,1.68,0.04,76.50,58.12,254.14,256.43,9.06,14.72,1.00,1.00,1.00
std,0.91,0.19,9.13,7.22,54.76,55.57,2.02,3.62,0.00,0.00,0.00
min,0.00,0.00,52.00,38.00,10.00,10.00,2.90,6.00,1.00,1.00,1.00
25%,1.12,0.00,70.00,53.00,260.00,270.00,8.10,12.10,1.00,1.00,1.00
50%,1.57,0.00,76.00,58.00,270.00,270.00,8.90,14.10,1.00,1.00,1.00
75%,2.01,0.00,83.00,64.00,270.00,280.00,10.10,16.10,1.00,1.00,1.00
max,8.05,2.67,111.00,79.00,360.00,360.00,21.00,34.00,1.00,1.00,1.00


In [49]:
df.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08
0,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-01,2.46,None,0.00,None,64,43,10.00,30.00,8.10,11.00,nan,nan,1.00
1,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-02,2.01,None,0.00,None,65,47,270.00,30.00,6.00,8.90,nan,nan,nan
2,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-03,0.67,None,0.00,None,62,44,150.00,150.00,10.10,14.10,nan,nan,nan
3,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-04,1.34,None,0.01,None,69,55,270.00,280.00,8.10,14.10,nan,nan,nan
4,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-05,2.46,None,1.61,None,59,49,140.00,140.00,10.10,16.10,1.00,1.00,nan


### Creating the compute target
Here we will define the compute cluster for our runs

In [18]:
cpu_cluster_name = 'EduardoCluster'
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [50]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'normalized_root_mean_squared_error'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target='EduardoCluster',
                             task = "regression",
                             training_data=dataset,
                             label_column_name="PRCP", 
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [51]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Capstone_ML,AutoML_0f49481a-6cc2-4b54-a242-070a63178de7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [52]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

We can retrieve the best model from the autoML run



In [53]:
best_autoML_run, best_autmoML_model = remote_run.get_output()

From the properties we can see that the best model was a Voting Ensemble using the following algorithms:

- 'ExtremeRandomTrees', 
-  4 'XGBoostRegressor'
- 'DecisionTree'

In [54]:
best_autoML_run.get_metrics()

{'r2_score': 0.46857357653418524,
 'mean_absolute_percentage_error': 145.7127459404339,
 'normalized_root_mean_squared_error': 0.050807436783423955,
 'normalized_root_mean_squared_log_error': nan,
 'normalized_mean_absolute_error': 0.0141147129958378,
 'root_mean_squared_log_error': nan,
 'explained_variance': 0.4686239398764736,
 'mean_absolute_error': 0.03768628369888693,
 'root_mean_squared_error': 0.13565585621174195,
 'normalized_median_absolute_error': 0.00046186491024953523,
 'median_absolute_error': 0.0012331793103662592,
 'spearman_correlation': 0.3868281056582576,
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_0f49481a-6cc2-4b54-a242-070a63178de7_38/residuals',
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_0f49481a-6cc2-4b54-a242-070a63178de7_38/predicted_true'}

In [55]:
best_autoML_run.get_details()

{'runId': 'AutoML_0f49481a-6cc2-4b54-a242-070a63178de7_38',
 'target': 'EduardoCluster',
 'status': 'Completed',
 'startTimeUtc': '2022-01-27T08:53:28.704951Z',
 'endTimeUtc': '2022-01-27T08:54:51.094928Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'regression\',\'primary_metric\':\'normalized_root_mean_squared_error\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'Capstone_ML\',\'compute_target\':\'EduardoCluster\',\'subscription_id\':\'2c48c51c-bd47-40d4-abbe-fb8eabd19c8c\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_0f49481a-6cc2-4b54-a242-070a63178de7_38","experiment_name":"Capstone_ML","workspace_name":"quick-starts-ws-179058","subs

We then proceed to register the model for later deployment

In [56]:
#TODO: Save the best model
autoML_model = best_autoML_run.register_model(model_name = 'best_autoML_model', model_path = './outputs/model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [57]:
from azureml.core import Model
from azureml.core.conda_dependencies import CondaDependencies

environment = best_autoML_run.get_environment()

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


# we retrieve the score script generated with the registered model
save_path = 'score.py'
best_autoML_run.download_file('outputs/scoring_file_v_1_0_0.py', save_path)

inference_config = InferenceConfig(entry_script = save_path, environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=2,description = 'precipitation predicter for LA')

service_name = 'precipitation-predictor'
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[autoML_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)


In [58]:
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-01-27 08:56:05+00:00 Creating Container Registry if not exists.
2022-01-27 08:56:05+00:00 Registering the environment.
2022-01-27 08:56:05+00:00 Use the existing image.
2022-01-27 08:56:05+00:00 Generating deployment configuration.
2022-01-27 08:56:06+00:00 Submitting deployment to compute.
2022-01-27 08:56:09+00:00 Checking the status of deployment precipitation-predictor..
2022-01-27 09:02:13+00:00 Checking the status of inference endpoint precipitation-predictor.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [59]:
print('Service REST URI: {}'.format(service.scoring_uri))
print('Swagger URI: {}'.format(service.swagger_uri))

Service REST URI: http://aca0c789-96cb-49f3-b38b-e1b01e398b5a.southcentralus.azurecontainer.io/score
Swagger URI: http://aca0c789-96cb-49f3-b38b-e1b01e398b5a.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [136]:
import requests
import json

# We get a sample from the original dataframe
df = dataset.to_pandas_dataframe()
data_sample = df.sample(1).fillna(0)
precipitation_sample = data_sample.pop('PRCP')

# Transform the data sample into a json format
dataload = json.dumps({'data': data_sample.to_dict(orient='records')})
headers = {'Content-type': 'application/json'}


# Send a post request to the service
dataload = '{"data": [{"AWND": 5.59,"TMAX": 71,"TMIN": 53,"WDF2": 310.00,"WDF5": 310.00,"WSF2": 16.10,"WSF5": 25.90,"WT01": 0.00,"WT02": 0.00,"WT08": 0.00}]}'
post_request = requests.post(service.scoring_uri,dataload,headers)
post_request.text

'{"message": "Expects Content-Type to be application/json"}'

In [137]:
dataload = '{"data": [{"AWND": 5.59,"TMAX": 71,"TMIN": 53,"WDF2": 310.00,"WDF5": 310.00,"WSF2": 16.10,"WSF5": 25.90,"WT01": 0.00,"WT02": 0.00,"WT08": 0.00}]}'

post_request = requests.post(service.scoring_uri,dataload,headers)
post_request.text



'{"message": "Expects Content-Type to be application/json"}'

In [88]:
df = dataset.to_pandas_dataframe()
data_sample = df.sample(1).fillna(0)
precipitation_sample = data_sample.pop('PRCP')

# Transform the data sample into a json format
dataload = json.dumps({'data': data_sample.to_dict(orient='records')})
print(dataload)

{"data": [{"AWND": 1.79, "TMAX": 83, "TMIN": 60, "WDF2": 260.0, "WDF5": 270.0, "WSF2": 8.9, "WSF5": 13.0, "WT01": 0.0, "WT02": 0.0, "WT08": 1.0}]}


In [84]:
data_sample

,AWND,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08
81,5.59,71,53,310.00,310.00,16.10,25.90,0.00,0.00,1.00


In [119]:
post_request.text

'{"message": "Expects Content-Type to be application/json"}'

In [61]:
data_sample

,AWND,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08
1432,0.45,69,54,90.00,120.00,6.00,8.10,nan,nan,nan
22,1.12,64,54,350.00,360.00,8.90,15.00,1.00,nan,1.00
1310,1.57,87,66,280.00,270.00,8.90,14.10,nan,nan,1.00
1191,0.89,70,56,270.00,270.00,12.10,19.00,nan,nan,nan


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
dir()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
